# Object Detection: Coffee Mugs, Pens and Computer Monitors
Below we will do the following:
1. Setup the training environment.
2. Load images of coffee mugs, pens and computer monitors and their corresponding bounding boxes.
2. Build an object detection model for identifying the location of coffee mugs, pens and computer monitors in images.
3. Convert the model to CoreML and download it.

The example is based on [Turi Create's Object Detection model](https://github.com/apple/turicreate/tree/master/userguide/object_detection).

## Environment Setup
Below we ensure `CUDA 10` is installed and then use pip to install `turicreate` and `mxnet-cu100` libraries.

In [ ]:
!pip install turicreate==5.4
# The wrong version of mxnet will be installed
!pip uninstall -y mxnet
# Install CUDA10-compatible version of mxnet
!pip install mxnet-cu100

## Data Preparation and Model Training
The data for this example are 186 images: 100 coffee mugs, 47 computer monitors, and 39 pens. The original data was taken from Google's open source [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html).

Once loaded, the data is randomly split into train and test sets, where 80% of the data is used for training and 20% is used for model evaluation (if you desire). Training this model with a GPU is much faster than CPU time. By default, this runtime environment should be using a Python 3 GPU backend instance. Below, we tell Turicreate to use all available GPUs for processing.

In [ ]:
# Import libraries and tell Turicreate to use all GPUs available
import tarfile
import urllib.request

import coremltools
import turicreate as tc
tc.config.set_num_gpus(-1)

In [ ]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ObjectDetection/coffee_pen_computer.sframe.tar.gz"
data_path = "coffee_pen_computer.sframe.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [ ]:
## Load the data
data =  tc.SFrame('./coffee_pen_computer.sframe')

## Make a train-test split
train_data, test_data = data.random_split(0.8)

In [ ]:
train_data.head()

In [ ]:
# Create an object detection model
model = tc.object_detector.create(
    dataset=train_data
)

# Object Detector Training Docs:
# https://apple.github.io/turicreate/docs/api/generated/turicreate.object_detector.create.html#turicreate.object_detector.create

## Model Export and Download
We convert the model to CoreML format so that it can run on an iOS device. Then we download it locally so it can be delivered to your apps with Skafos.

In [ ]:
# Specify the CoreML model name
model_name = 'ObjectDetection'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 


In [ ]:
# Download the model you just trained!! This may take a few moments
from google.colab import files
files.download(coreml_model_name)